# Вебинар 5. Ранжирование и Гибридные рекоммендательные системы

# 1. Ранжирование

### 1.1 BPR - Bayesian Personalized Ranking loss

$$loss = \sigma(p^{+} - p^{-})$$

1. Случайная пара user - купленный им item_i = позитивная пара
2. Для того же юзера сэмплируем item_j (не купил или вес в user-item матрице ниже, у item_i из 1.) - негативная пара
3. Прогнозы модели $p^{+}$, $p^{-}$ --> $p^{+} - p^{-}$
4. loss = $\sigma(p^{+} - p^{-})$ - это и есть наша ошибка
5. Обновляем эмбеддинги

### 1.2 WARP = Weighted Approximate-Rank Pairwise loss
Неплохое [объяснение](https://medium.com/@gabrieltseng/intro-to-warp-loss-automatic-differentiation-and-pytorch-b6aa5083187a) на игрушечном примере

$$loss = ln(\frac{K-1}{N})*(p^- - p^+)$$

1. Сэмплируем случайную позитивную пару юзер-item, прогноз модели для этой пары $p^+$
2. Сюмплируем далее до тех пор, пока не найдем негативную пару, гду прогнозная вероятность вероятность $p^-$ > $p^+$
3. K - кол-во сэмплирований (зависит от качества модели!)
4. N - общее количество рекомендаций

**Note**  
loss зависит только от 2ух сэмплированных пар, эмбеддинги только ждя этих пар будут обновлены

In [1]:
# Case 1
p_plus = 4.3  # true = 1
p_neg_1 = -2.3
p_neg_2 = 4.5

loss(random_seed) = [(3-1) / 10 * (4.5 - 4.3)]

# Case 2 -  модель лучше в этом случае
p_plus = 4.3  # true = 1
p_neg_1 = -2.3
p_neg_2 = 0.5
p_neg_3 = -0.5
p_neg_4 = 2.1
p_neg_5 = 4.5

loss(random_seed) = [(6-1) / 10 * (4.5 - 4.3)]

SyntaxError: ignored

### Зачем нужно ранжирование? Есть же классификация (предсказание 0 и 1)
- Ранжирование гораздо лучше выбирает топ-k товаров, особенно когда k велико
- Ранжирование позволяет напрямуб ранжировать 5 релевантных юзеру товаров (какой из 5 поставить выше?)

НО:
- При ранжировании невозможно использовать предсказание как вероятность

Прогноз (p) = вероятность  

Мат ож выручки = p * Price

---

# 2. LightFM

![mf.png](attachment:mf.png)

Формула эмбеддинга в ALS (матричной факторизации)

$$user_{emb} = \theta_1 * i_1 + ... + \theta_n * i_n$$  
  
$$p = user_{emb} * item_{emb}^T + b_u + b_i$$

Формула в LightFM

$$user_{emb} = \theta_1 * i_1 + ... + \theta_n * i_n + \alpha_1 * f_1 + ... + \alpha_k * f_k$$  
  
$$p = user_{emb} * item_{emb}^T + b_u + b_i$$

### Зачем нужны гибридные системы?

*Холодный старт* - ситуация, в которой с товаром/юзером в матрице мало взаимодействий

Холодный страт для товаров - обычно введение нового товара (YouTube, Lamoda, hh.ru)  
Холодный старт для юзера - к нам пришел новый юзер  

Холодного юзера/товар можно описать фичами --> мы больше о нем знаем --> он уже не такой холодный

In [90]:
!pip install lightfm

In [4]:
!pip install implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 45.8 MB/s eta 0:00:00


In [88]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)


In [55]:
data = pd.read_csv('retail_train.csv')
item_features = pd.read_csv('product.csv')
user_features = pd.read_csv('hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# train test split
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [56]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [57]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


In [58]:
user_features['age_desc'].unique()

array(['65+', '45-54', '25-34', '35-44', '19-24', '55-64'], dtype=object)

In [59]:
user_features['marital_status_code'].unique()

array(['A', 'U', 'B'], dtype=object)

In [60]:
user_features['household_size_desc'].unique()

array(['2', '3', '4', '1', '5+'], dtype=object)

In [61]:
def prefilter_items(data, item_features):

    n_items_before = data['item_id'].nunique()

    # Уберем самые популярные товары (их и так купят)
    popularity = data.groupby('item_id')['user_id'].nunique().reset_index() / data_train['user_id'].nunique() #вычисляем популярность товаров (сколько пользователей купило / общее кол-во пользователей)
    popularity.rename(columns={'user_id': 'share_unique_users'}, inplace=True)

    top_popular = popularity[popularity['share_unique_users'] > 0.5].item_id.tolist() #берем список топ-товаров с популярностью > 0.5
    data = data[~data['item_id'].isin(top_popular)] # убираем из данных транзакции по топ-товарам

    n_items_after = data['item_id'].nunique()
    print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

    # Уберем самые НЕ популярные товары (их и так НЕ купят)
    top_notpopular = popularity[popularity['share_unique_users'] < 0.01].item_id.tolist() #берем список НЕтоп-товаров с популярностью < 0.01
    data = data[~data['item_id'].isin(top_notpopular)] # убираем из данных транзакции по НЕтоп-товарам

    n_items_after = data['item_id'].nunique()
    print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

    # Уберем товары, которые не продавались за последние 12 месяцев (52 недели)
    data = data[data['week_no'] > 50]

    n_items_after = data['item_id'].nunique()
    print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

    # Уберем не интересные для рекоммендаций категории (department)
    category = pd.DataFrame(item_features.groupby('department')['item_id'].count().sort_values(ascending=False)).reset_index()
    category.columns = ['department', 'count_items']
    category = category[category['count_items'] > 150]

    top_category_items = item_features[item_features['department'].isin(category)].item_id.tolist()
    data = data[~data['item_id'].isin(top_notpopular)]

    n_items_after = data['item_id'].nunique()
    print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

    # Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб.
    data['price'] = data['sales_value'] / data['quantity']
    data = data[data['price'] > 2]

    n_items_after = data['item_id'].nunique()
    print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

    # Уберем товары с нулем в quantity
    data = data[~(data['quantity'] == 0)]

    n_items_after = data['item_id'].nunique()
    print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

    # Уберем слишком дорогие товары
    data = data[data['price'] <= 50]

    n_items_after = data['item_id'].nunique()
    print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

    return data

## 1. Filter items

In [62]:
n_items_before = data_train['item_id'].nunique()

data_train = prefilter_items(data_train, item_features)

n_items_after = data_train['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 86865 to 86865
Decreased # items from 86865 to 86865
Decreased # items from 86865 to 60247
Decreased # items from 86865 to 60247
Decreased # items from 86865 to 40356
Decreased # items from 86865 to 40333
Decreased # items from 86865 to 40281
Decreased # items from 86865 to 40281


## 2. Prepare data set

In [63]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
# data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train,
                                  index='user_id', columns='item_id',
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count',
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(2)

item_id,25671,26636,26889,27764,27812,27951,27978,28018,28041,28158,...,17328345,17328742,17329473,17329749,17330255,17330511,17382205,17383227,17827644,17829232
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [64]:
data_test = data_test[data_test['item_id'].isin(data_train['item_id'].unique())]
test_user_item_matrix = pd.pivot_table(data_test,
                                  index='user_id', columns='item_id',
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count',
                                  fill_value=0
                                 )

test_user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

In [65]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [67]:
user_item_matrix.index

Int64Index([   1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
            ...
            2491, 2492, 2493, 2494, 2495, 2496, 2497, 2498, 2499, 2500],
           dtype='int64', name='user_id', length=2449)

## 3. Prepare user and item features

In [66]:
user_feat = pd.DataFrame(user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)

item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)

user_feat.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
user_id,,,,,,,
1,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [68]:
item_feat.head(2)

,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
item_id,,,,,,
25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
26636,69,PASTRY,Private,BREAKFAST SWEETS,SW GDS: SW ROLLS/DAN,


In [69]:
user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())

In [70]:
user_feat_lightfm.head(2)

,age_desc_19-24,age_desc_25-34,age_desc_35-44,age_desc_45-54,age_desc_55-64,age_desc_65+,marital_status_code_A,marital_status_code_B,marital_status_code_U,income_desc_100-124K,...,hh_comp_desc_Unknown,household_size_desc_1,household_size_desc_2,household_size_desc_3,household_size_desc_4,household_size_desc_5+,kid_category_desc_1,kid_category_desc_2,kid_category_desc_3+,kid_category_desc_None/Unknown
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,1,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [71]:
item_feat_lightfm.head(5)

,manufacturer_2,manufacturer_4,manufacturer_5,manufacturer_6,manufacturer_8,manufacturer_11,manufacturer_13,manufacturer_15,manufacturer_16,manufacturer_17,...,curr_size_of_product_SIZE 2,curr_size_of_product_SIZE 3,curr_size_of_product_SM/MED,curr_size_of_product_SMALL,curr_size_of_product_SNGL,curr_size_of_product_SOFT,curr_size_of_product_SZ1 2PK,curr_size_of_product_TALL,curr_size_of_product_UNDER 15LB,curr_size_of_product_X-LARGE
item_id,,,,,,,,,,,,,,,,,,,,,
25671,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26636,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26889,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27764,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27812,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
model = LightFM(no_components=30,
                loss='bpr', # 'warp'
                learning_rate=0.05,
                item_alpha=0.1, user_alpha=0.1,
                random_state=42)

model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=coo_matrix(user_item_matrix),
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          epochs=15,
          num_threads=4)

In [73]:
user_emb = model.get_user_representations(features=csr_matrix(user_feat_lightfm.values).tocsr())

In [74]:
user_emb[0]  # biases

array([-4.6040006,  0.       ,  0.       , ..., -4.708548 , -3.1227047,
        0.       ], dtype=float32)

In [75]:
user_emb[0].shape

(2449,)

In [76]:
user_emb[1]  # embeddings

array([[ 7.9322336e-18,  3.4323649e-18,  1.1097461e-17, ...,
         9.9413825e-18, -2.8521221e-17,  4.5878539e-18],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       ...,
       [-1.8490101e-18,  1.7750957e-18,  4.9123758e-18, ...,
        -8.4790951e-18,  1.6318309e-19,  4.4403643e-18],
       [-1.1415803e-17,  2.9357560e-18,  7.3520963e-18, ...,
        -7.6809237e-18,  1.1148045e-17, -2.2261159e-18],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00]], dtype=float32)

In [77]:
user_emb[1].shape

(2449, 30)

In [78]:
item_emb = model.get_item_representations(features=csr_matrix(item_feat_lightfm.values).tocsr())

In [79]:
item_emb[0].shape

(40281,)

In [80]:
item_emb[1].shape

(40281, 30)

In [94]:
train_precision = precision_at_k(model, sparse_user_item,
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()

train_precision

0.0

In [92]:
test_item_ids = np.array([1, 2, 3, 200, 1200])

predictions = model.predict(user_ids=0, item_ids=test_item_ids,
                            user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                            item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                            num_threads=4)

In [93]:
predictions

array([-4.6127872, -4.60106  , -4.614209 , -4.6052003, -4.6183243],
      dtype=float32)

predictions - некоторые числа.